In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("./../..")

In [3]:
from pathlib import Path
import math
import pickle
#
import torch
import torchvision
from torchvision import utils
import torchvision.transforms as T
import torchvision.datasets as datasets
#
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm
#
from misc.plot_utils import plot_mat, imshow
from effcn.functions import max_norm_masking
from effcn.models_mnist import Backbone, Decoder, EffCapsNet, CNN_CR_SF, CNN_CR, CNN_R

### Preprocessing

In [4]:
device = torch.device("cuda")
device

device(type='cuda')

In [5]:
p_experiment = "/mnt/data/experiments/EfficientCN/mnist/effcn_mnist_CNN_R_2022_02_01_19_25_57"
p_experiment = Path(p_experiment)
p_ckpts = p_experiment / "ckpts"
p_data = Path("/mnt/data/datasets")
p_model = p_ckpts / "model_150.ckpt"
p_model.exists()

False

In [7]:
model = MnistEffCapsNet()
#model = MnistCNN_R()
model.load_state_dict(torch.load(p_model))
model = model.to(device)
model.eval()

NameError: name 'MnistEffCapsNet' is not defined

In [ ]:
ds_train = datasets.MNIST(root=p_data, train=True, download=True)#, transform=T.ToTensor(), target_transform=T.ToTensor())
ds_valid = datasets.MNIST(root=p_data, train=False, download=True)#, transform=None, target_transform=T.ToTensor())
  

In [ ]:
idx = 2

print(len(ds_train.data))

x = ds_train.data[idx]
y = ds_train.targets[idx]



x = torch.unsqueeze(torch.unsqueeze(x,dim=0),dim=0).float()
y = torch.unsqueeze(y,dim=0)

print(x.size())
print(y.size(), y)

### Generate Capsuls for influenced Reconstruction

In [ ]:
x = x.to(device)
uh, _ = model.forward(x)

In [ ]:
print(uh.size())

In [ ]:
m_uh = max_norm_masking(uh)
m_uh = torch.flatten(m_uh, start_dim=1)
x_rec = model.decoder(m_uh)

In [ ]:
def imshow(img, cmap="gray", vmin=None, vmax=None):
    npimg = img.detach().cpu().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap=cmap, vmin=None, vmax=None)
    plt.show()

In [ ]:
imshow(torch.squeeze(x_rec,dim=0))

In [ ]:
print(torch.argmax(uh,dim=2))

sq_uh = torch.squeeze(uh,dim=0)
a = torch.norm(sq_uh, dim=1)
plt.bar(np.arange(0,10,1), a.detach().cpu().numpy())
plt.show()

i_cap = torch.argmax(torch.norm(uh, dim=2))
ref = uh[:,i_cap,:]

uh_delta = uh.clone()
uh_delta[:,i_cap,:] += torch.tensor([0.1,0,-0.1,0,0,0,0.1,0,0,0,0,0,0,0,0,0]).to(device)
delta = uh_delta[:,i_cap,:]

torch.manual_seed(42)
uh_delta2 = uh.clone()
uh_delta2[:,i_cap,:] += (torch.rand(16).to(device)-0.5) * 0.1
delta2 = uh_delta2[:,i_cap,:]

print(torch.squeeze(ref,dim=0))
plt.plot(torch.squeeze(ref,dim=0).detach().cpu().numpy(),"b*")
print(torch.squeeze(delta,dim=0))
plt.plot(torch.squeeze(delta,dim=0).detach().cpu().numpy(),"r*")
print(torch.squeeze(delta2,dim=0))
plt.plot(torch.squeeze(delta2,dim=0).detach().cpu().numpy(),"g*")
plt.show()




uh_n = torch.cat((uh, uh_delta, uh_delta2), dim=0)  

uh_n.size()

#print(uh_delta-uh)


In [ ]:
m_uh_n = max_norm_masking(uh_n)
m_uh_n = torch.flatten(m_uh_n, start_dim=1)
x_rec_n = model.decoder(m_uh_n)

In [ ]:
print(x_rec_n.size())
rec = x_rec_n.cpu()
scal = lambda x: (x-x.min())/(x.max()-x.min())
img = torchvision.utils.make_grid(torch.cat([scal(rec)], dim=0), nrow=rec.shape[0])
plt.imshow(img.permute(1,2,0))
plt.show()

### Prepare affine transformations embedding

In [ ]:
x_aff = T.functional.affine(img=x, angle=0, translate=[0,0], scale=1.,shear=0)

imshow(torch.squeeze(x_aff,dim=0))

In [ ]:
def affine_xtrans(img, target, range=[-5.,5.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    x_trans = torch.zeros([len(arange),img.shape[1],img.shape[2],img.shape[3]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        x_trans[i] = T.functional.affine(img=x, angle=0, translate=[l,0], scale=1.,shear=0)
        l_target[i] = target
    
    return x_trans, l_target

def affine_ytrans(img, target, range=[-5.,5.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    y_trans = torch.zeros([len(arange),img.shape[1],img.shape[2],img.shape[3]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        y_trans[i] = T.functional.affine(img=x, angle=0, translate=[0,l], scale=1.,shear=0)
        l_target[i] = target
    
    return y_trans, l_target

def affine_rot(img, target, range=[-25.,25.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    rot = torch.zeros([len(arange),img.shape[1],img.shape[2],img.shape[3]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        rot[i] = T.functional.affine(img=x, angle=l, translate=[0,0], scale=1.,shear=0)
        l_target[i] = target
    
    return rot, l_target

def affine_scale(img, target, range=[0.75,1.25,0.05]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    rot = torch.zeros([len(arange),img.shape[1],img.shape[2],img.shape[3]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        rot[i] = T.functional.affine(img=x, angle=0, translate=[0,0], scale=l,shear=0)
        l_target[i] = target
    
    return rot, l_target


def affine_shear(img, target, range=[-10.,10.,2]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    rot = torch.zeros([len(arange),img.shape[1],img.shape[2],img.shape[3]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        rot[i] = T.functional.affine(img=x, angle=0, translate=[0,0], scale=1,shear=l)
        l_target[i] = target
    
    return rot, l_target

In [ ]:
x_trams, y_trans = affine_shear(x,y)
print(x_trams.shape)
print(y_trans)

x_trams = x_trams.cpu()
scal = lambda x: (x-x.min())/(x.max()-x.min())
img = torchvision.utils.make_grid(torch.cat([scal(x_trams)], dim=0), nrow=x_trams.shape[0])
plt.imshow(img.permute(1,2,0))
plt.show()

imshow(x_trams[0,:,:,:])


In [ ]:
x_trans, _ = affine_xtrans(x,y)
y_trans, _ = affine_ytrans(x,y)
rot, _ = affine_rot(x,y)

### ATE on x_trans

In [ ]:
x_aff = x_trans.to(device)
#x_aff = y_trans.to(device)
#x_aff = rot.to(device)

uh_aff, _ = model.forward(x_aff)

In [ ]:
def cov_uh_trans(uh):
    """
    uh in [k,n,m]
    k -> number of transformed images
    n -> number of output classes
    m -> number of capsul values
    """

    uh_mean = uh.mean(dim=0)
    """
    #could be cleaner, but it's the same ...
    uh_mean = uh_mean.unsqueeze(0)
    uh_mean = uh_mean.repeat(uh.shape[0],1,1)
    """

    z = uh - uh_mean

    c_k = torch.einsum('...ij, ...ik -> ...jk', z,z)
    c = torch.einsum('ijk -> jk', c_k) / c_k.shape[0]

    return c

In [ ]:
c = cov_uh_trans(uh_aff)


In [ ]:
def covshow(c):
    # https://matplotlib.org/3.1.1/gallery/images_contours_and_fields/image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py

    data = c.cpu().detach().numpy()

    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(data)

    fg_color = 'white'
    im.axes.tick_params(color=fg_color, labelcolor=fg_color)

    """
    # Loop over data dimensions and create text annotations.
    for i in range(len(data)):
        for j in range(len(data)):
            text = ax.text(j, i, data[i, j],
                        ha="center", va="center", color="fg_color")
    """

    cb = fig.colorbar(im)
    cb.ax.yaxis.set_tick_params(color=fg_color)
    cb.outline.set_edgecolor(fg_color)
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color=fg_color)
    fig.tight_layout()
    plt.show()

In [ ]:
covshow(c)

### via PCA from Paper EffCN

In [ ]:
cov = c.cpu()

eig, v_eig = torch.linalg.eig(cov)
eig = eig.float()
sig = eig / eig.sum()

#rint(eig)
#print(eig.sum())
print(sig)
plt.plot(sig.detach().numpy(),".")
plt.tick_params(colors="w")



### ! wrong (Via KL-Divergende from munich paper)

In [ ]:
### !!! Not shure about this!!!

# https://mail.python.org/pipermail/scipy-user/2011-May/029521.html

def KLdivergence(x, y):
  """Compute the Kullback-Leibler divergence between two multivariate samples.
  Parameters
  ----------
  x : 2D array (n,d)
    Samples from distribution P, which typically represents the true
    distribution.
  y : 2D array (m,d)
    Samples from distribution Q, which typically represents the approximate
    distribution.
  Returns
  -------
  out : float
    The estimated Kullback-Leibler divergence D(P||Q).
  References
  ----------
  Pérez-Cruz, F. Kullback-Leibler divergence estimation of
continuous distributions IEEE International Symposium on Information
Theory, 2008.
  """
  from scipy.spatial import cKDTree as KDTree

  # Check the dimensions are consistent
  x = np.atleast_2d(x)
  y = np.atleast_2d(y)

  n,d = x.shape
  m,dy = y.shape

  assert(d == dy)


  # Build a KD tree representation of the samples and find the nearest neighbour
  # of each point in x.
  xtree = KDTree(x)
  ytree = KDTree(y)

  # Get the first two nearest neighbours for x, since the closest one is the
  # sample itself.
  r = xtree.query(x, k=2, eps=.01, p=2)[0][:,1]
  s = ytree.query(x, k=1, eps=.01, p=2)[0]

  # There is a mistake in the paper. In Eq. 14, the right side misses a negative sign
  # on the first term of the right hand side.
  return -np.log(r/s).sum() * d / n + np.log(m / (n - 1.))

In [ ]:
def cov_uh_kl(uh):
    c_k = torch.einsum('...ij, ...ik -> ...jk', uh,uh)
    c = torch.einsum('ijk -> jk', c_k) / c_k.shape[0]
    return c 

In [ ]:
c_org = cov_uh_kl(uh.cpu())
c_aff = cov_uh_kl(uh_aff.cpu())

#determinaten
c_org_det = torch.linalg.det(c_org)
c_aff_det = torch.linalg.det(c_aff)

#trace
c_org_det = torch.trace(c_org)
c_aff_det = torch.trace(c_aff)

#compute kl div from git hub
kl_div = KLdivergence(c_org.detach().numpy(), c_aff.detach().numpy())

print(kl_div)
covshow(c_org)
covshow(c_aff)

### Via KL-Divergende from munich paper

In [ ]:
print(uh.shape)
print(uh_aff.shape)

get vectors from groud truth

In [ ]:
print(y)

uh_th = uh[:,y,:]
uh_aff_th = uh_aff[:,y,:]
print(uh_th.shape)
print(uh_aff_th.shape)

uh_aff_var = torch.var(uh_aff_th, dim=0)
print(uh_aff_var)
print(uh_aff_var.shape)


uh_aff_sum = uh_aff_var.sum()
uh_aff_nor = uh_aff_var / uh_aff_sum
print(uh_aff_sum)
print(uh_aff_nor)


uni_p = 1/uh_aff_nor.shape[1]
uni_p

In [ ]:
print(uh_aff_nor.sum())
print(uni_p*16)

In [ ]:
(uh_aff_nor * torch.log((uh_aff_nor/uni_p)))

In [ ]:
(uh_aff_nor * torch.log((uh_aff_nor/uni_p))).sum()

Multi Image proceccing

In [ ]:
ds_train = datasets.MNIST(root=p_data, train=True, download=True, transform=T.ToTensor())
ds_valid = datasets.MNIST(root=p_data, train=False, download=True, transform=None)

dl_train = torch.utils.data.DataLoader(ds_train, 
                                        #batch_size=len(ds_train), 
                                        batch_size=64, 
                                        shuffle=False)

dl_valid = torch.utils.data.DataLoader(ds_valid, 
                                        batch_size=64, 
                                        shuffle=False)

In [ ]:
x,y = next(iter(dl_train))

print(x.shape)
print(y.shape)

In [ ]:
def affine_xtrans(img, target, range=[-5.,5.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    x_trans = torch.zeros([len(arange),img.shape[-3],img.shape[-2],img.shape[-1]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        x_trans[i] = T.functional.affine(img=img, angle=0, translate=[l,0], scale=1.,shear=0)
        l_target[i] = target
    
    return x_trans, l_target

def affine_ytrans(img, target, range=[-5.,5.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    y_trans = torch.zeros([len(arange),img.shape[-3],img.shape[-2],img.shape[-1]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        y_trans[i] = T.functional.affine(img=img, angle=0, translate=[0,l], scale=1.,shear=0)
        l_target[i] = target
    
    return y_trans, l_target

def affine_rot(img, target, range=[-25.,25.,1]):
    arange = np.arange(range[0],(range[1]+range[2]),range[2])
    rot = torch.zeros([len(arange),img.shape[-3],img.shape[-2],img.shape[-1]])
    l_target = torch.zeros(len(arange))

    for i, l in enumerate(arange):
        rot[i] = T.functional.affine(img=img, angle=l, translate=[0,0], scale=1.,shear=0)
        l_target[i] = target
    
    return rot, l_target
   

In [ ]:
def cov_uh_trans(uh):
    """
    uh in [k,n,m]
    k -> number of transformed images
    n -> number of output classes
    m -> number of capsul values
    """

    uh_mean = uh.mean(dim=0)
    z = uh - uh_mean
    c_k = torch.einsum('...ij, ...ik -> ...jk', z,z)
    c = torch.einsum('ijk -> jk', c_k) / c_k.shape[0]

    return c

In [ ]:
affine = affine_xtrans
pca_eig = []
kl_div = []

pbar = tqdm(dl_train, bar_format='{bar:10}{r_bar}{bar:-10b}')

#load batchwise
for x, y in pbar:
    #calculate staistical vals
    for i, img in enumerate(x):
        #generate aff transforms
        x_aff, y_aff = affine(x[i],y[i])
        
        #Generate Caps from affine Transform
        x_aff = x_aff.to(device)
        uh_aff, _ = model.forward(x_aff)

        #PCA
        #Covariance from Caps
        cov_uh = cov_uh_trans(uh_aff)
        #Eigenvals
        eig, v_eig = torch.linalg.eig(cov_uh)
        sig = eig.float() / eig.float().sum()
        #PCA eigenvalues
        pca_eig.append(sig.tolist())


        #KL-Divergence
        #Caps from valid
        uh_aff_th = uh_aff[:,y[i],:]
        #Variance over each dimension
        var_uh_aff = torch.var(uh_aff_th, dim=0)
        #Variance normalized
        nor_uh_aff = var_uh_aff / var_uh_aff.sum()
        #uniform prior
        uni_p = 1/nor_uh_aff.shape[0]
        #Kullback-Leibler-Divergenz
        kl = (nor_uh_aff * torch.log((nor_uh_aff/uni_p))).sum()
        kl_div.append(kl.tolist())


pca_mean = torch.tensor(pca_eig).mean(dim=0)
kld_mean = torch.tensor(kl_div).mean()

In [ ]:
print(min(kl_div))
print(kld_mean)
print(pca_mean)